# Lab 7

In [16]:
from load_data import load_data
import numpy as np
from scipy import linalg

# Load data and print the available keys for the data

In [17]:
data = load_data()

In [18]:
print(data.keys())

dict_keys(['dataMat1', 'ITest', 'VTest', 'meanTest', 'vecTest', 'mTest', 'vEigenTest1', 'vEigenTest2', 'faceDetection'])


# Task 1

In [134]:
def get_eigenvectors(dataMat, k):
    # np.cov calculates the covariance matrix of x
    covMat = np.cov(dataMat.T)
    n = len(covMat)
    eigenvalues, eigenvectors = linalg.eigh(covMat, subset_by_index = [n - k, n - 1] )
    # Reverse the order of elements in an array along the given axis.
    eigenvectors = np.flip(eigenvectors, axis = 1)
    print(eigenvectors)

    return eigenvectors

def test(expected, args, method):
    act = method(*args)
    comparison = act == expected
    if not comparison.all():
        print(f'test failed for method {method.__name__}()\nexpected\t{expected}\ngot\t\t{act}')
    elif comparison.all():
        print(f'test successful')


# Test case 1
test_case_1 = np.array([[0.6779, -0.7352], [0.7352, 0.6779]])
test(test_case_1, [data['dataMat1'], 2], get_eigenvectors)

# Test case 2
test_case_2 = np.array([[0.6779, -0.7352]])
test(test_case_2, [data['dataMat1'], 1], get_eigenvectors)


[[ 0.6778734  -0.73517866]
 [ 0.73517866  0.6778734 ]]
test failed for method get_eigenvectors()
expected	[[ 0.6779 -0.7352]
 [ 0.7352  0.6779]]
got		[[ 0.6778734  -0.73517866]
 [ 0.73517866  0.6778734 ]]
[[0.6778734 ]
 [0.73517866]]
test failed for method get_eigenvectors()
expected	[[ 0.6779 -0.7352]]
got		[[0.6778734 ]
 [0.73517866]]


# Task 2

In [135]:
def get_mean(dataMat):
    m = dataMat.mean(axis=0)
    return m

def test(expected, args, method):
    act = method(*args)
    comparison = act == expected
    if not comparison.all():
        print(f'test failed for method {method.__name__}()\nexpected\t{expected}\ngot\t\t{act}')
    elif comparison.all():
        print(f'test successful')

# Test case 1
test(np.array([1.8100, 1.9100]), [data['dataMat1']], get_mean)

test failed for method get_mean()
expected	[1.81 1.91]
got		[1.81 1.91]


# Task 3

In [136]:
def compressPCAVector(vec, m, vEigen):
    # Centered vector vec - m, substract the mean
    vec_transformed = vEigen.T@(vec - m)
    vec_compressed = (vEigen@vec_transformed) + m
    return vec_compressed

def test(expected, args, method):
    act = method(*args)
    comparison = act == expected
    if not comparison.all():
        print(f'test failed for method {method.__name__}()\nexpected\t{expected}\ngot\t\t{act}')
    elif comparison.all():
        print(f'test successful')

# Test case 1
test(np.array([0.60493717, 0.60307834]), [data['vecTest'], data['mTest'], data['vEigenTest1']], compressPCAVector)

# Test case 2
test(np.array([0.49991164, 0.69991839]), [data['vecTest'], data['mTest'], data['vEigenTest2']], compressPCAVector)


test failed for method compressPCAVector()
expected	[0.60493717 0.60307834]
got		[0.60493717 0.60307834]
test failed for method compressPCAVector()
expected	[0.49991164 0.69991839]
got		[0.49991164 0.69991839]


# Task 4

In [171]:
def transformingImageToVector(M):
    V = M.reshape((M.shape[0]*M.shape[1]), 1)

    return V, [M.shape[0], M.shape[1]]

def transformingVectorToImage(V, height, width):
    M = V.reshape(height, width)

    return M

def compressPCAImage(I, nEigenvectors, V, mean):
    V1, shape_image = transformingImageToVector(I)
    # mean is also a image and must be transformed to a vector
    mean_vector = transformingImageToVector(mean)[0]
    vEigen = V[:, :nEigenvectors]
    ICompressed = compressPCAVector(V1, mean_vector, vEigen)
    ICompressed = transformingVectorToImage(ICompressed, shape_image[0], shape_image[1])

    return ICompressed

# Test case 1
nEigenvectors = 4
# ICompressed
mean = get_mean(data['VTest'])
ICompressed = compressPCAImage(data['ITest'], nEigenvectors, data['VTest'], data['meanTest'] )
print(ICompressed)

[[2141. 2144. 3267.]
 [3376. 2569. 4262.]]


# Task 5